In [1]:
import tensorflow as tf

In [3]:
#tf.train.BytesList/FloatList/Int64List

In [2]:
# movie name and ratings for a single user
movie_name_list = tf.train.BytesList(value=[b'The Shawshank Redemption', b'Fight club'])
movie_rating_list = tf.train.FloatList(value=[9.0, 9.7])

In [4]:
#tf.train.Feature

In [5]:
movie_names = tf.train.Feature(bytes_list=movie_name_list)
movie_ratings = tf.train.Feature(float_list=movie_rating_list)

In [6]:
#tf.train.Features <= a collection of named features

In [7]:
movie_dict = {
    'Movie Names': movie_names, #name: feature
    'Movie Ratings': movie_ratings
}
movies = tf.train.Features(feature=movie_dict)

In [8]:
# tf.train.Example

In [9]:
example = tf.train.Example(features=movies)

In [10]:
with tf.python_io.TFRecordWriter('movie_ratings.tfrecord') as writer:
    writer.write(example.SerializeToString())

In [11]:
### FULL EXAMPLE ###

In [12]:
data = {
    'Age': 29,
    'Movie': ['The Shawshank Redemption', 'Fight Club'],
    'Movie Ratings': [9.0, 9.7],
    'Suggestion': 'Inception',
    'Suggestion Purchased': 1.0,
    'Purchase Price': 9.99
}
print(data)


{'Age': 29, 'Movie': ['The Shawshank Redemption', 'Fight Club'], 'Movie Ratings': [9.0, 9.7], 'Suggestion': 'Inception', 'Suggestion Purchased': 1.0, 'Purchase Price': 9.99}


In [16]:
#Create the Example
example = tf.train.Example(
    features = tf.train.Features( 
        feature = {
            'Age': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['Age']])),
            'Movie': tf.train.Feature(bytes_list = tf.train.BytesList(value=[m.encode('utf-8') for m in data['Movie']])),
            'Movie Ratings': tf.train.Feature(float_list=tf.train.FloatList(value=data['Movie Ratings'])),
            'Suggestion': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['Suggestion'].encode('utf-8')])),
            'Suggestion Purchased': tf.train.Feature(float_list = tf.train.FloatList(value=[data['Suggestion Purchased']])),
            'Purchased Price': tf.train.Feature(float_list=tf.train.FloatList(value=[data['Purchase Price']]))
    })
)

print(example)

features {
  feature {
    key: "Age"
    value {
      int64_list {
        value: 29
      }
    }
  }
  feature {
    key: "Movie"
    value {
      bytes_list {
        value: "The Shawshank Redemption"
        value: "Fight Club"
      }
    }
  }
  feature {
    key: "Movie Ratings"
    value {
      float_list {
        value: 9.0
        value: 9.699999809265137
      }
    }
  }
  feature {
    key: "Purchased Price"
    value {
      float_list {
        value: 9.989999771118164
      }
    }
  }
  feature {
    key: "Suggestion"
    value {
      bytes_list {
        value: "Inception"
      }
    }
  }
  feature {
    key: "Suggestion Purchased"
    value {
      float_list {
        value: 1.0
      }
    }
  }
}



In [17]:
# Write record to file 

In [18]:
with tf.python_io.TFRecordWriter('customer_1.tfrecord') as writer:
    # write accepts a string as parameter, therefore structured data must be serialized first.
    writer.write(example.SerializeToString())

In [19]:
# Read data from file and print

In [24]:
sess = tf.InteractiveSession()

reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(['customer_1.tfrecord'])

_, serialized_example = reader.read(filename_queue)

# Define features
read_features = {
    'Age': tf.FixedLenFeature([], dtype=tf.int64),
    'Movie': tf.VarLenFeature(dtype=tf.string),
    'Movie Ratings': tf.VarLenFeature(dtype=tf.float32),
    'Suggestion': tf.FixedLenFeature([], dtype=tf.string),
    'Suggestion Purchased': tf.FixedLenFeature([], dtype=tf.float32),
    'Purchased Price': tf.FixedLenFeature([], dtype=tf.float32)
}

#Extract features from serialized data
read_data = tf.parse_single_example(serialized=serialized_example, features=read_features)
tf.train.start_queue_runners(sess)

for name, tensor in read_data.items():
    print("{}: {}".format(name, tensor.eval()))

Movie: SparseTensorValue(indices=array([[0],
       [1]]), values=array([b'The Shawshank Redemption', b'Fight Club'], dtype=object), dense_shape=array([2]))
Movie Ratings: SparseTensorValue(indices=array([[0],
       [1]]), values=array([9. , 9.7], dtype=float32), dense_shape=array([2]))
Age: 29
Purchased Price: 9.989999771118164
Suggestion: b'Inception'
Suggestion Purchased: 1.0


/Users/danielobembe/miniconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
